# Phân tích Bán hàng Nâng cao - Bài 2: Phân tích Doanh thu và Khách hàng

Notebook này thực hiện phân tích chi tiết về doanh thu, khách hàng và xu hướng bán hàng.

## 🔧 Thiết lập môi trường

In [ ]:
# Import thư viện
import sys
sys.path.append('/app')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import database helper
from db_connection import get_db_connection

# Thiết lập style
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✅ Đã thiết lập môi trường thành công!")

## 🔌 Kết nối Database

In [ ]:
# Kết nối database
db = get_db_connection()
print("✅ Đã kết nối thành công đến SQL Server!")

## 🏆 Phân tích Khách hàng VIP

In [ ]:
# Query phân tích khách hàng chi tiết
customer_analysis_query = """
SELECT 
    c.CustomerID,
    c.CustomerName,
    c.City,
    c.JoinDate,
    COUNT(DISTINCT o.OrderID) as SoLuongDonHang,
    SUM(od.Quantity) as TongSoLuongSanPham,
    SUM(od.Quantity * p.Price) as TongGiaTriMuaHang,
    AVG(od.Quantity * p.Price) as GiaTriTrungBinhMoiDon,
    MIN(o.OrderDate) as DonHangDauTien,
    MAX(o.OrderDate) as DonHangCuoiCung,
    DATEDIFF(day, MIN(o.OrderDate), MAX(o.OrderDate)) as SoNgayMuaHang
FROM Customers c
LEFT JOIN Orders o ON c.CustomerID = o.CustomerID
LEFT JOIN OrderDetails od ON o.OrderID = od.OrderID
LEFT JOIN Products p ON od.ProductID = p.ProductID
GROUP BY c.CustomerID, c.CustomerName, c.City, c.JoinDate
ORDER BY TongGiaTriMuaHang DESC
"""

customer_analysis = db.query_to_dataframe(customer_analysis_query)
print("🏆 PHÂN TÍCH KHÁCH HÀNG VIP")
print("=" * 60)
display(customer_analysis)

# Thống kê tổng quan
print(f"\n📊 THỐNG KÊ TỔNG QUAN:")
print(f"💰 Tổng doanh thu: ${customer_analysis['TongGiaTriMuaHang'].sum():.2f}")
print(f"📦 Tổng số đơn hàng: {customer_analysis['SoLuongDonHang'].sum()}")
print(f"🛍️ Tổng số sản phẩm bán: {customer_analysis['TongSoLuongSanPham'].sum()}")
print(f"💵 Giá trị trung bình mỗi khách hàng: ${customer_analysis['TongGiaTriMuaHang'].mean():.2f}")

In [ ]:
# Biểu đồ phân tích khách hàng
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Doanh thu theo khách hàng
sns.barplot(data=customer_analysis, x='TongGiaTriMuaHang', y='CustomerName', ax=axes[0,0])
axes[0,0].set_title('💰 Doanh thu theo khách hàng', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Tổng giá trị mua hàng ($)')

# 2. Số đơn hàng theo khách hàng
sns.barplot(data=customer_analysis, x='SoLuongDonHang', y='CustomerName', ax=axes[0,1], color='orange')
axes[0,1].set_title('📦 Số đơn hàng theo khách hàng', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Số lượng đơn hàng')

# 3. Doanh thu theo thành phố
city_revenue = customer_analysis.groupby('City')['TongGiaTriMuaHang'].sum().sort_values(ascending=False)
axes[1,0].pie(city_revenue.values, labels=city_revenue.index, autopct='%1.1f%%', startangle=90)
axes[1,0].set_title('🏙️ Doanh thu theo thành phố', fontsize=14, fontweight='bold')

# 4. Mối quan hệ giữa số đơn hàng và giá trị
scatter = axes[1,1].scatter(customer_analysis['SoLuongDonHang'], 
                           customer_analysis['TongGiaTriMuaHang'],
                           c=customer_analysis['GiaTriTrungBinhMoiDon'], 
                           cmap='viridis', s=100, alpha=0.7)
axes[1,1].set_xlabel('Số lượng đơn hàng')
axes[1,1].set_ylabel('Tổng giá trị mua hàng ($)')
axes[1,1].set_title('📈 Mối quan hệ: Số đơn hàng vs Giá trị', fontsize=14, fontweight='bold')
plt.colorbar(scatter, ax=axes[1,1], label='Giá trị TB/đơn ($)')

# Thêm tên khách hàng vào scatter plot
for i, txt in enumerate(customer_analysis['CustomerName']):
    axes[1,1].annotate(txt, (customer_analysis['SoLuongDonHang'].iloc[i], 
                            customer_analysis['TongGiaTriMuaHang'].iloc[i]),
                      xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.tight_layout()
plt.show()

## 🛍️ Phân tích Sản phẩm Bán chạy

In [ ]:
# Query phân tích sản phẩm chi tiết
product_analysis_query = """
SELECT 
    p.ProductID,
    p.ProductName,
    p.Category,
    p.Price,
    COALESCE(SUM(od.Quantity), 0) as TongSoLuongBan,
    COALESCE(SUM(od.Quantity * p.Price), 0) as TongDoanhThu,
    COALESCE(COUNT(DISTINCT od.OrderID), 0) as SoLuongDonHang,
    COALESCE(AVG(CAST(od.Quantity as FLOAT)), 0) as SoLuongTrungBinhMoiDon,
    COALESCE(COUNT(DISTINCT o.CustomerID), 0) as SoKhachHangMua
FROM Products p
LEFT JOIN OrderDetails od ON p.ProductID = od.ProductID
LEFT JOIN Orders o ON od.OrderID = o.OrderID
GROUP BY p.ProductID, p.ProductName, p.Category, p.Price
ORDER BY TongDoanhThu DESC
"""

product_analysis = db.query_to_dataframe(product_analysis_query)
print("🛍️ PHÂN TÍCH SẢN PHẨM BÁN CHẠY")
print("=" * 60)
display(product_analysis)

# Thống kê theo danh mục
category_stats = product_analysis.groupby('Category').agg({
    'TongDoanhThu': 'sum',
    'TongSoLuongBan': 'sum',
    'ProductID': 'count'
}).rename(columns={'ProductID': 'SoLuongSanPham'})

print(f"\n📂 THỐNG KÊ THEO DANH MỤC:")
display(category_stats)

In [ ]:
# Biểu đồ phân tích sản phẩm
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Top sản phẩm theo doanh thu
top_revenue = product_analysis.head(5)
sns.barplot(data=top_revenue, x='TongDoanhThu', y='ProductName', ax=axes[0,0])
axes[0,0].set_title('💰 Top 5 sản phẩm theo doanh thu', fontweight='bold')
axes[0,0].set_xlabel('Doanh thu ($)')

# 2. Top sản phẩm theo số lượng bán
top_quantity = product_analysis.nlargest(5, 'TongSoLuongBan')
sns.barplot(data=top_quantity, x='TongSoLuongBan', y='ProductName', ax=axes[0,1], color='orange')
axes[0,1].set_title('📦 Top 5 sản phẩm theo số lượng', fontweight='bold')
axes[0,1].set_xlabel('Số lượng bán')

# 3. Doanh thu theo danh mục
axes[0,2].pie(category_stats['TongDoanhThu'].values, 
              labels=category_stats.index, 
              autopct='%1.1f%%', startangle=90)
axes[0,2].set_title('📂 Doanh thu theo danh mục', fontweight='bold')

# 4. Mối quan hệ giá và số lượng bán
scatter = axes[1,0].scatter(product_analysis['Price'], 
                           product_analysis['TongSoLuongBan'],
                           c=product_analysis['TongDoanhThu'], 
                           cmap='plasma', s=100, alpha=0.7)
axes[1,0].set_xlabel('Giá ($)')
axes[1,0].set_ylabel('Số lượng bán')
axes[1,0].set_title('💲 Giá vs Số lượng bán', fontweight='bold')
plt.colorbar(scatter, ax=axes[1,0], label='Doanh thu ($)')

# 5. Số lượng bán theo danh mục
sns.barplot(data=category_stats.reset_index(), x='Category', y='TongSoLuongBan', ax=axes[1,1])
axes[1,1].set_title('📊 Số lượng bán theo danh mục', fontweight='bold')
axes[1,1].set_xlabel('Danh mục')
axes[1,1].set_ylabel('Tổng số lượng bán')
axes[1,1].tick_params(axis='x', rotation=45)

# 6. Hiệu suất bán hàng (Doanh thu / Số khách hàng)
product_analysis['HieuSuatBanHang'] = product_analysis['TongDoanhThu'] / (product_analysis['SoKhachHangMua'] + 1)
top_efficiency = product_analysis.nlargest(5, 'HieuSuatBanHang')
sns.barplot(data=top_efficiency, x='HieuSuatBanHang', y='ProductName', ax=axes[1,2], color='green')
axes[1,2].set_title('⚡ Top 5 hiệu suất bán hàng', fontweight='bold')
axes[1,2].set_xlabel('Doanh thu / Khách hàng ($)')

plt.tight_layout()
plt.show()

## 📅 Phân tích Xu hướng Thời gian

In [ ]:
# Query phân tích theo thời gian
time_analysis_query = """
SELECT 
    o.OrderDate,
    COUNT(DISTINCT o.OrderID) as SoLuongDonHang,
    COUNT(DISTINCT o.CustomerID) as SoKhachHangMua,
    SUM(od.Quantity) as TongSoLuongSanPham,
    SUM(od.Quantity * p.Price) as DoanhThuNgay,
    AVG(od.Quantity * p.Price) as GiaTriTrungBinhDon,
    AVG(od.Quantity) as SoLuongTrungBinhMoiDon
FROM Orders o
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
GROUP BY o.OrderDate
ORDER BY o.OrderDate
"""

time_analysis = db.query_to_dataframe(time_analysis_query)
print("📅 PHÂN TÍCH XU HƯỚNG THEO THỜI GIAN")
print("=" * 60)
display(time_analysis)

# Tính toán các chỉ số tăng trưởng
time_analysis['DoanhThuLuyKe'] = time_analysis['DoanhThuNgay'].cumsum()
time_analysis['DonHangLuyKe'] = time_analysis['SoLuongDonHang'].cumsum()

print(f"\n📈 TỔNG KẾT XU HƯỚNG:")
print(f"💰 Tổng doanh thu: ${time_analysis['DoanhThuNgay'].sum():.2f}")
print(f"📦 Tổng đơn hàng: {time_analysis['SoLuongDonHang'].sum()}")
print(f"📊 Doanh thu trung bình/ngày: ${time_analysis['DoanhThuNgay'].mean():.2f}")
print(f"📊 Đơn hàng trung bình/ngày: {time_analysis['SoLuongDonHang'].mean():.1f}")

In [ ]:
# Biểu đồ xu hướng thời gian
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Doanh thu theo ngày
axes[0,0].plot(time_analysis['OrderDate'], time_analysis['DoanhThuNgay'], 
               marker='o', linewidth=2, markersize=8, color='blue')
axes[0,0].set_title('💰 Doanh thu theo ngày', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Ngày')
axes[0,0].set_ylabel('Doanh thu ($)')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)

# 2. Số đơn hàng theo ngày
axes[0,1].plot(time_analysis['OrderDate'], time_analysis['SoLuongDonHang'], 
               marker='s', linewidth=2, markersize=8, color='orange')
axes[0,1].set_title('📦 Số đơn hàng theo ngày', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Ngày')
axes[0,1].set_ylabel('Số đơn hàng')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(True, alpha=0.3)

# 3. Doanh thu lũy kế
axes[1,0].plot(time_analysis['OrderDate'], time_analysis['DoanhThuLuyKe'], 
               marker='^', linewidth=3, markersize=8, color='green')
axes[1,0].set_title('📈 Doanh thu lũy kế', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Ngày')
axes[1,0].set_ylabel('Doanh thu lũy kế ($)')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)

# 4. Giá trị trung bình đơn hàng
axes[1,1].bar(time_analysis['OrderDate'], time_analysis['GiaTriTrungBinhDon'], 
              color='red', alpha=0.7, width=0.8)
axes[1,1].set_title('💵 Giá trị trung bình đơn hàng', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Ngày')
axes[1,1].set_ylabel('Giá trị TB ($)')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 Phân đoạn Khách hàng (Customer Segmentation)

In [ ]:
# Tạo phân đoạn khách hàng dựa trên RFM (Recency, Frequency, Monetary)
# Recency: Thời gian từ lần mua cuối
# Frequency: Số lần mua hàng
# Monetary: Tổng giá trị mua hàng

# Tính toán ngày hiện tại (giả sử là ngày sau cùng trong dữ liệu + 1)
max_date = time_analysis['OrderDate'].max()
current_date = max_date + timedelta(days=1)

# Tính Recency (số ngày từ lần mua cuối)
customer_analysis['Recency'] = (current_date - customer_analysis['DonHangCuoiCung']).dt.days
customer_analysis['Frequency'] = customer_analysis['SoLuongDonHang']
customer_analysis['Monetary'] = customer_analysis['TongGiaTriMuaHang']

# Tạo điểm số RFM (1-5, 5 là tốt nhất)
customer_analysis['R_Score'] = pd.qcut(customer_analysis['Recency'].rank(method='first'), 5, labels=[5,4,3,2,1])
customer_analysis['F_Score'] = pd.qcut(customer_analysis['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
customer_analysis['M_Score'] = pd.qcut(customer_analysis['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])

# Tạo RFM Score tổng hợp
customer_analysis['RFM_Score'] = (customer_analysis['R_Score'].astype(str) + 
                                 customer_analysis['F_Score'].astype(str) + 
                                 customer_analysis['M_Score'].astype(str))

# Phân loại khách hàng
def segment_customers(row):
    if row['RFM_Score'] in ['555', '554', '544', '545', '454', '455', '445']:
        return 'Champions'
    elif row['RFM_Score'] in ['543', '444', '435', '355', '354', '345', '344', '335']:
        return 'Loyal Customers'
    elif row['RFM_Score'] in ['512', '511', '422', '421', '412', '411', '311']:
        return 'New Customers'
    elif row['RFM_Score'] in ['155', '154', '144', '214', '215', '115', '114']:
        return 'At Risk'
    else:
        return 'Others'

customer_analysis['Segment'] = customer_analysis.apply(segment_customers, axis=1)

print("🎯 PHÂN ĐOẠN KHÁCH HÀNG (RFM ANALYSIS)")
print("=" * 60)
segment_summary = customer_analysis.groupby('Segment').agg({
    'CustomerID': 'count',
    'TongGiaTriMuaHang': ['sum', 'mean'],
    'SoLuongDonHang': 'mean',
    'Recency': 'mean'
}).round(2)

segment_summary.columns = ['SoLuongKH', 'TongDoanhThu', 'DoanhThuTB', 'DonHangTB', 'RecencyTB']
display(segment_summary)

print("\n📊 Chi tiết phân đoạn khách hàng:")
display(customer_analysis[['CustomerName', 'City', 'RFM_Score', 'Segment', 'TongGiaTriMuaHang']].sort_values('TongGiaTriMuaHang', ascending=False))

In [ ]:
# Biểu đồ phân đoạn khách hàng
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Phân bố khách hàng theo segment
segment_counts = customer_analysis['Segment'].value_counts()
axes[0,0].pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%', startangle=90)
axes[0,0].set_title('👥 Phân bố khách hàng theo phân đoạn', fontsize=14, fontweight='bold')

# 2. Doanh thu theo segment
segment_revenue = customer_analysis.groupby('Segment')['TongGiaTriMuaHang'].sum().sort_values(ascending=True)
axes[0,1].barh(segment_revenue.index, segment_revenue.values, color='lightblue')
axes[0,1].set_title('💰 Doanh thu theo phân đoạn', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Tổng doanh thu ($)')

# 3. RFM Score distribution
sns.scatterplot(data=customer_analysis, x='Recency', y='Monetary', 
                hue='Segment', size='Frequency', ax=axes[1,0], alpha=0.7)
axes[1,0].set_title('📊 Phân tích RFM: Recency vs Monetary', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Recency (ngày)')
axes[1,0].set_ylabel('Monetary ($)')

# 4. Frequency vs Monetary by Segment
sns.scatterplot(data=customer_analysis, x='Frequency', y='Monetary', 
                hue='Segment', ax=axes[1,1], s=100, alpha=0.7)
axes[1,1].set_title('📈 Frequency vs Monetary theo phân đoạn', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Frequency (số đơn hàng)')
axes[1,1].set_ylabel('Monetary ($)')

plt.tight_layout()
plt.show()

## 💾 Lưu kết quả phân tích

In [ ]:
# Lưu kết quả phân tích
customer_analysis.to_csv('/app/data/customer_analysis_advanced.csv', index=False)
product_analysis.to_csv('/app/data/product_analysis_advanced.csv', index=False)
time_analysis.to_csv('/app/data/time_analysis.csv', index=False)
segment_summary.to_csv('/app/data/customer_segments.csv')

print("✅ Đã lưu tất cả kết quả phân tích vào thư mục /app/data/")
print("📁 Các file đã tạo:")
print("   - customer_analysis_advanced.csv")
print("   - product_analysis_advanced.csv")
print("   - time_analysis.csv")
print("   - customer_segments.csv")

# Đóng kết nối
db.close()
print("\n🔌 Đã đóng kết nối database")
print("✅ Hoàn thành phân tích bán hàng nâng cao!")